In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# File paths for Forest and Grassland datasets
forest_file = "/content/drive/MyDrive/CSV/Bird_Monitoring_Data_FOREST.XLSX"
grassland_file = "/content/drive/MyDrive/CSV/Bird_Monitoring_Data_GRASSLAND.XLSX"

# Function to read all sheets from an Excel file, clean and combine into one DataFrame
def read_and_combine_sheets(file_path, habitat_type):
    # Read all sheets into a dictionary of DataFrames
    sheets = pd.read_excel(file_path, sheet_name=None)
    # Remove columns that are completely empty (all NaN) to avoid issues
    cleaned_sheets = [df.dropna(axis=1, how='all') for df in sheets.values()]
    # Concatenate all sheets vertically
    df = pd.concat(cleaned_sheets, ignore_index=True)
    # Add a column indicating habitat type (Forest or Grassland)
    df["habitat_type"] = habitat_type
    return df

# Read and combine forest and grassland data
df_forest = read_and_combine_sheets(forest_file, "Forest")
df_grassland = read_and_combine_sheets(grassland_file, "Grassland")

# Merge both datasets into one DataFrame
df = pd.concat([df_forest, df_grassland], ignore_index=True)

# Standardize column names: strip whitespace, lowercase, replace spaces with underscores
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")

# Handle missing values
for col in df.columns:
    if df[col].dtype == "object":
        # For categorical/text columns, fill missing with mode (most frequent value)
        mode_val = df[col].mode(dropna=True)[0]
        df[col] = df[col].fillna(mode_val)
    else:
        # For numeric columns, fill missing with median to avoid distortion by outliers
        median_val = df[col].median()
        df[col] = df[col].fillna(median_val)

# Convert 'date' column to datetime type and create month and season columns if 'date' exists
if "date" in df.columns:
    df["date"] = pd.to_datetime(df["date"], errors="coerce")  # Convert invalid dates to NaT
    df["month"] = df["date"].dt.month
    # Map months to seasons
    df["season"] = df["month"].map({
        12: "Winter", 1: "Winter", 2: "Winter",
        3: "Spring", 4: "Spring", 5: "Spring",
        6: "Summer", 7: "Summer", 8: "Summer",
        9: "Autumn", 10: "Autumn", 11: "Autumn"
    })

# Placeholder for distance conversion if needed (example: add distance_m column)
if "distance" in df.columns and "distance_m" not in df.columns:
    df["distance_m"] = None

# Save the cleaned and merged dataset to CSV
output_file = "/content/drive/MyDrive/CSV/merged_clean_bird_data.csv"
df.to_csv(output_file, index=False)
print(f"Cleaned and merged dataset saved as {output_file}")

# ------------------- Visualization Section -------------------

# Chart 1: Top 10 Most Observed Bird Species
plt.figure(figsize=(10,6))
sns.countplot(
    y='common_name', 
    data=df, 
    order=df['common_name'].value_counts().iloc[:10].index,
    palette="viridis"
)
plt.title("Top 10 Most Observed Bird Species")
plt.xlabel("Observation Count")
plt.ylabel("Bird Species")
plt.show()

# Chart 2: Temperature vs. Humidity Scatter Plot colored by Season
plt.figure(figsize=(8,6))
sns.scatterplot(
    data=df, 
    x="temperature", 
    y="humidity", 
    hue="season", 
    alpha=0.7, 
    palette="coolwarm"
)
plt.title("Temperature vs. Humidity by Season")
plt.xlabel("Temperature (°C)")
plt.ylabel("Humidity (%)")
plt.show()

# Chart 3: Flyover Bird Observations Count
plt.figure(figsize=(6,5))
sns.countplot(x="flyover_observed", data=df, palette="pastel")
plt.title("Flyover Bird Observations")
plt.xlabel("Flyover Observed")
plt.ylabel("Count")
plt.show()

# Chart 4: Top 10 Observers by Observation Count
plt.figure(figsize=(10,6))
top_observers = df["observer"].value_counts().head(10)
sns.barplot(x=top_observers.values, y=top_observers.index, palette="magma")
plt.title("Top 10 Observers by Number of Observations")
plt.xlabel("Observation Count")
plt.ylabel("Observer")
plt.show()

# Chart 5: Sex Distribution of Observed Birds
plt.figure(figsize=(7,5))
sns.countplot(x="sex", data=df)
plt.title("Sex Distribution of Observed Birds")
plt.xlabel("Sex")
plt.ylabel("Number of Observations")
plt.show()

# Chart 6: Observation Counts by Identification Method
plt.figure(figsize=(8,5))
sns.countplot(x="id_method", data=df, order=df["id_method"].value_counts().index)
plt.title("Observation Counts by Identification Method")
plt.xlabel("Identification Method")
plt.ylabel("Number of Observations")
plt.show()

# Chart 7: Observations by Distance Category
plt.figure(figsize=(8,5))
sns.countplot(x="distance", data=df, order=df["distance"].value_counts().index)
plt.title("Observations by Distance Category")
plt.xlabel("Distance")
plt.ylabel("Number of Observations")
plt.show()

# Chart 8: Temperature Distribution During Bird Observations
plt.figure(figsize=(8,5))
sns.histplot(df["temperature"], bins=20, kde=True, color="skyblue")
plt.title("Temperature Distribution During Bird Observations")
plt.xlabel("Temperature (°C)")
plt.ylabel("Frequency")
plt.show()

# Chart 9: Humidity Distribution During Bird Observations
plt.figure(figsize=(8,5))
sns.histplot(df["humidity"], bins=20, kde=True, color="lightgreen")
plt.title("Humidity Distribution During Bird Observations")
plt.xlabel("Humidity (%)")
plt.ylabel("Frequency")
plt.show()

# Chart 10: Observations by Sky Condition
plt.figure(figsize=(8,5))
sns.countplot(x="sky", data=df, order=df["sky"].value_counts().index)
plt.title("Observations by Sky Condition")
plt.xlabel("Sky Condition")
plt.ylabel("Number of Observations")
plt.xticks(rotation=45)
plt.show()

# Chart 11: Observations by Wind Condition
plt.figure(figsize=(8,5))
sns.countplot(x="wind", data=df, order=df["wind"].value_counts().index)
plt.title("Observations by Wind Condition")
plt.xlabel("Wind Condition")
plt.ylabel("Number of Observations")
plt.xticks(rotation=45)
plt.show()

# Chart 12: PIF Watchlist Species Observations
plt.figure(figsize=(6,5))
sns.countplot(x="pif_watchlist_status", data=df)
plt.title("PIF Watchlist Species Observations")
plt.xlabel("On PIF Watchlist")
plt.ylabel("Number of Observations")
plt.show()

# Chart 13: Number of Unique Bird Species per Season
plt.figure(figsize=(8,6))
season_species = df.groupby("season")["common_name"].nunique().reset_index()
sns.barplot(x="season", y="common_name", data=season_species, palette="Set2")
plt.title("Number of Unique Bird Species per Season")
plt.xlabel("Season")
plt.ylabel("Unique Species Count")
plt.show()

# Correlation Heatmap for Numeric Features
plt.figure(figsize=(10,6))
sns.heatmap(df.corr(numeric_only=True), annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Correlation Heatmap")
plt.show()

# Boxplot of Temperature Distribution by Season
plt.figure(figsize=(8,5))
sns.boxplot(x="season", y="temperature", data=df)
plt.title("Temperature Distribution by Season")
plt.xlabel("Season")
plt.ylabel("Temperature (°C)")
plt.show()
